Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cobra

from src.gem.yeast8model import Yeast8Model
from src.data.biomasscomponent import biomass_component_list_orig
from src.calc.flux import compare_fluxes


# Construct models of cells of interest, optimise

In [ ]:
#glucose_bounds = (-1, 0) # nearer the default
glucose_bounds = (-4.75, 0) # gives a sensible growth rate for wt
#glucose_bounds = (-18.5, 0) # for E. coli

wt = Yeast8Model("../data/gemfiles/yeast-GEM_8-6-0.xml", growth_id="r_2111", biomass_id="r_4041")
wt.biomass_component_list = biomass_component_list_orig
wt.add_media_components(['r_1992'])
wt.model.reactions.r_1714.bounds = glucose_bounds
wt.solution = wt.optimize()

BY4741 = Yeast8Model("../data/gemfiles/yeast-GEM_8-6-0.xml", growth_id="r_2111", biomass_id="r_4041")
BY4741.biomass_component_list = biomass_component_list_orig
BY4741.add_media_components(['r_1992'])
BY4741.model.reactions.r_1714.bounds = glucose_bounds
BY4741.make_auxotroph("BY4741")
BY4741.solution = BY4741.optimize()

zwf1 = Yeast8Model("../data/gemfiles/yeast-GEM_8-6-0.xml", growth_id="r_2111", biomass_id="r_4041")
zwf1.biomass_component_list = biomass_component_list_orig
zwf1.add_media_components(['r_1992'])
zwf1.model.reactions.r_1714.bounds = glucose_bounds
zwf1.make_auxotroph("BY4741")
zwf1.knock_out_list(["YNL241C"])
zwf1.solution = zwf1.optimize()

tsa2 = Yeast8Model("../data/gemfiles/yeast-GEM_8-6-0.xml", growth_id="r_2111", biomass_id="r_4041")
tsa2.biomass_component_list = biomass_component_list_orig
tsa2.add_media_components(['r_1992'])
tsa2.model.reactions.r_1714.bounds = glucose_bounds
tsa2.make_auxotroph("BY4742")
tsa2.knock_out_list(["YDR453C"])
tsa2.solution = tsa2.optimize()

# Ablate

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
wt.ablation_barplot(ax)
plt.show()

In [ ]:
BY4741.ablation_result = BY4741.ablate()

fig, ax = plt.subplots()
BY4741.ablation_barplot(ax)
plt.show()

> ~~Predicts unrealistically rapid synthesis time for BY strains, but this may be because of infinity uptake of media supplements needed for these auxotrophs to grow~~

In [ ]:
zwf1.ablation_result = zwf1.ablate()
fig, ax = plt.subplots()
zwf1.ablation_barplot(ax)
plt.show()

In [ ]:
tsa2.ablation_result = tsa2.ablate()
fig, ax = plt.subplots()
tsa2.ablation_barplot(ax)
plt.show()

In [ ]:
wt.ablation_result

In [ ]:
BY4741.ablation_result

In [ ]:
zwf1.ablation_result

In [ ]:
tsa2.ablation_result

# Compare fluxes

In [ ]:
diff_fluxes_sorted = compare_fluxes(BY4741, zwf1)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
diff_fluxes_sorted = compare_fluxes(BY4741, tsa2)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, BY4741)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
for reaction_id in BY4741.model.medium.keys():
    print(BY4741.model.reactions.get_by_id(reaction_id).name)

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, BY4741)

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, wt)

In [ ]:
diff_fluxes_sorted

# Glucose uptake

In [ ]:
from src.calc.growth import get_exch_saturation

In [ ]:
# positive values because i want it increasing in a plot
glc_exch_rates = np.linspace(0, 1000, 100)
wt_growthrates = get_exch_saturation(wt, "r_1714", glc_exch_rates)

In [ ]:
fig, ax = plt.subplots()
ax.plot(glc_exch_rates, wt_growthrates, label='prototroph (wild type)')
ax.set_xlim((0,1000))
ax.set_xlabel('Glucose exchange rate (mmol/gDW/h)')
ax.set_ylabel('Growth rate (/h)')
ax.set_title('Effect of glucose exchange rate on growth rate')
ax.legend()

In [ ]:
# Get exchange rate
np.interp(0.41, wt_growthrates, glc_exch_rates)

# Pyruvate uptake

In [ ]:
# positive values because i want it increasing in a plot
pyr_exch_rates = np.linspace(0, 1000, 100)
wt_growthrates = get_exch_saturation(wt, "r_2033", pyr_exch_rates)

In [ ]:
fig, ax = plt.subplots()
ax.plot(pyr_exch_rates, wt_growthrates, label='prototroph (wild type)')
ax.set_xlim((0,1000))
ax.set_xlabel('Pyruvate exchange rate (mmol/gDW/h)')
ax.set_ylabel('Growth rate (/h)')
ax.set_title('Effect of pyruvate exchange rate on growth rate')
ax.legend()

In [ ]:
# Get exchange rate
np.interp(0.41, wt_growthrates, pyr_exch_rates)

# Ammonium uptake

In [ ]:
nh4_exch_rates = np.linspace(0, 5, 100)
wt_growthrates = get_exch_saturation(wt, "r_1654", nh4_exch_rates, remove_glucose=False)

In [ ]:
fig, ax = plt.subplots()
ax.plot(nh4_exch_rates, wt_growthrates, label='prototroph (wild type)')
ax.set_xlim((0,5))
ax.set_xlabel('Ammonium exchange rate (mmol/gDW/h)')
ax.set_ylabel('Growth rate (/h)')
ax.set_title('Effect of ammonium exchange rate on growth rate')
ax.legend()

In [ ]:
# Get exchange rate
np.interp(0.40, wt_growthrates, nh4_exch_rates)